## Notebook to retrieve IWP from GMI TB and other auxiliary data using QRNN, uses quantNN

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData_v import gmiData
import os
import time
from quantnn.models.pytorch.fully_connected import FullyConnected
from torch import optim
from quantnn.qrnn import QRNN
from torch.optim import Adam
from quantnn.metrics import ScatterPlot
from quantnn.models.pytorch.logging import TensorBoardLogger
from quantnn.transformations import LogLinear, Log, Log10, Id


### Set Inputs

In [2]:
quantiles         = np.linspace(0.01, 0.99, 50)

#quantiles         =  np.array([0.2, 3 , 16 ,50 ,84 ,97, 99.8])

batchSize         = 256


n_layers          = 5
n_neurons         = 256


inputs            = np.array(["ta", "t2m",  "wvp", "z0", "lat", "stype"])
ninputs           = len(inputs) + 2 + 10

outputs           = "iwp"

latlims           = [0, 65]

filename          = "qrnn_gmi_nn_lpa_v1.nc"
outputfile        = os.path.join(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/"), 
                                                    filename)

## read in training data

In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_jan_lpa_pr1.nc"), 
               inputs, outputs, latlims = latlims, pratio = None,
               batch_size = batchSize, transform = None)  

norm = data.norm

validation_data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_test_jan_lpa_pr1.nc"), 
               inputs, outputs, latlims = latlims, normalise = norm, pratio = None,
               batch_size = batchSize, transform = None)  

n = len(data)
n_train = int(0.9 * n)
n_val = n - n_train

#training_data = DataLoader(data, batch_size=256, shuffle = True)

#validation_data = DataLoader(validation_data, batch_size=256, shuffle=True)

training_data, validation_data = random_split(data, [n_train, n_val])
results = []

print(len(validation_data))

training_data      = DataLoader(data, batch_size=None, num_workers=1, pin_memory=True)
validation_data    = DataLoader(validation_data, batch_size=None, num_workers=1, pin_memory=True)

387


In [4]:
data.x.shape

(988993, 18)

In [5]:
for i in ["0", "1", "2", "3"]:
    
    filename          = "qrnn_gmi_nn_lpa_pr1_v" + i + "_v.nc"
    outputfile        = os.path.join(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/"), 
                                                        filename)

    print(filename)
    skip_connections  = True
    batch_norm        = True

    skip_connections  = True
    batch_norm        = True
    transform         = LogLinear()

    model             = FullyConnected(ninputs,
                           quantiles.size,
                           n_layers,
                           n_neurons,
                           skip_connections=skip_connections,
                           batch_norm=batch_norm)

    #model = FullyConnected(n_inputs=ninputs, n_outputs=len(quantiles), n_layers=5, width=256)

    qrnn             = QRNN(quantiles, ninputs,  model, transformation = transform)

    metrics          = ["MeanSquaredError", "Bias", "CRPS", "CalibrationPlot"]

    n_epochs         = 200
    logger = TensorBoardLogger(n_epochs)

    scatter_plot    = ScatterPlot(bins=np.logspace(-2, 2, 21), log_scale=True)
    metrics.append(scatter_plot)

    for lr, n_epochs in zip([1e-2, 1e-3, 1e-4], [50, 50, 50]):

    #for lr, n_epochs in zip([0.01], [120]):

        #optimizer = Adam(qrnn.model.parameters(), lr=lr)

        optimizer = optim.SGD(model.parameters(), lr = lr, momentum = 0.9)

        #scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

        qrnn.train(training_data=training_data,
                   validation_data=validation_data,
                   n_epochs=n_epochs,
                   mask=-999,
                   device="cpu",
                   logger=logger,
                   metrics=metrics,
                   optimizer=optimizer, 
                   #scheduler=scheduler,
                  );

    qrnn.save(outputfile)

qrnn_gmi_nn_lpa_pr1_v0_v.nc


Output()

/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/in

                                                                                           
                                                                                           
                                      Training history                                     
                                                                                           
                                                                                           
       Epoch          Training loss      Validation loss      MSE       Bias       CRPS    
 ───────────────────────────────────────────────────────────────────────────────────────── 
     #      LR            Total               Total                                        
                                                                                           
      1   0.0100          0.758               0.649         0.058      -0.017    0.026     
      2   0.0100          0.651               0.639         0.044      -0.010    0.024     
      3   0.0100          0.643               0.632         0.040      -0.007    0.024     
      4   0.0100          0.638               0.628         0.038      -0.006    0.023     
      5   0.0100          0.635               0.624         0.036      -0.006    0.023     
      6   0.0100          0.632               0.622         0.035      -0.004    0.023     
      7   0.0100          0.629               0.620         0.035      -0.004    0.022     
      8   0.0100          0.626               0.617         0.033      -0.004    0.022     
      9   0.0100          0.624               0.614         0.033      -0.003    0.022     
     10   0.0100          0.622               0.612         0.032      -0.002    0.022     
     11   0.0100          0.620               0.610         0.032      -0.002    0.022     
     12   0.0100          0.618               0.608         0.031      -0.002    0.022     
     13   0.0100          0.616               0.607         0.031      -0.001    0.022     
     14   0.0100          0.615               0.606         0.031      -0.001    0.022     
     15   0.0100          0.613               0.605         0.030      0.001     0.022     
     16   0.0100          0.611               0.603         0.029      0.001     0.021     
     17   0.0100          0.610               0.602         0.029      -0.000    0.021     
     18   0.0100          0.608               0.601         0.028      -0.000    0.021     
     19   0.0100          0.607               0.599         0.028      0.000     0.021     
     20   0.0100          0.605               0.597         0.028      -0.000    0.021     
     21   0.0100          0.604               0.596         0.028      -0.001    0.021     
     22   0.0100          0.602               0.594         0.027      -0.000    0.021     
     23   0.0100          0.601               0.592         0.028      -0.002    0.021     
     24   0.0100          0.600               0.591         0.027      -0.001    0.021     
     25   0.0100          0.598               0.590         0.026      0.000     0.021     
     26   0.0100          0.597               0.589         0.026      -0.000    0.021     
     27   0.0100          0.596               0.588         0.026      0.001     0.021     
     28   0.0100          0.595               0.587         0.026      -0.000    0.021     
     29   0.0100          0.594               0.586         0.026      -0.001    0.020     
     30   0.0100          0.593               0.585         0.025      -0.001    0.021     
     31   0.0100          0.591               0.585         0.025      0.001     0.021     
     32   0.0100          0.590               0.582         0.025      -0.000    0.020     
     33   0.0100          0.589               0.583         0.024      0.000     0.020     
     34   0.0100          0.588               0.581         0.025      -0.000    0.020     
     35   0.0100          0.587             

Output()

...

                                                                                           
                                                                                           
                                      Training history                                     
                                                                                           
                                                                                           
       Epoch          Training loss      Validation loss      MSE       Bias       CRPS    
 ───────────────────────────────────────────────────────────────────────────────────────── 
     #      LR            Total               Total                                        
                                                                                           
     51   0.0010          0.554               0.554         0.019      0.000     0.019     
     52   0.0010          0.552               0.552         0.019      -0.000    0.018     
     53   0.0010          0.550               0.551         0.019      -0.001    0.018     
     54   0.0010          0.549               0.551         0.019      -0.001    0.018     
     55   0.0010          0.548               0.550         0.018      -0.001    0.018     
     56   0.0010          0.547               0.549         0.018      -0.001    0.018     
     57   0.0010          0.546               0.548         0.018      -0.001    0.018     
     58   0.0010          0.546               0.548         0.018      -0.001    0.018     
     59   0.0010          0.545               0.547         0.018      -0.001    0.018     
     60   0.0010          0.544               0.547         0.018      -0.001    0.018     
     61   0.0010          0.544               0.546         0.018      -0.001    0.018     
     62   0.0010          0.543               0.546         0.018      -0.001    0.018     
     63   0.0010          0.542               0.545         0.018      -0.001    0.018     
     64   0.0010          0.542               0.544         0.018      -0.001    0.018     
     65   0.0010          0.541               0.544         0.018      -0.001    0.018     
     66   0.0010          0.541               0.543         0.018      -0.001    0.018     
     67   0.0010          0.540               0.543         0.018      -0.000    0.018     
     68   0.0010          0.539               0.543         0.018      -0.000    0.018     
     69   0.0010          0.539               0.542         0.018      -0.000    0.018     
     70   0.0010          0.538               0.542         0.018      -0.000    0.018     
     71   0.0010          0.538               0.541         0.018      -0.000    0.018     
     72   0.0010          0.537               0.541         0.018      -0.000    0.018     
     73   0.0010          0.537               0.540         0.018      -0.000    0.018     
     74   0.0010          0.536               0.540         0.018      -0.000    0.018     
     75   0.0010          0.536               0.540         0.018      -0.001    0.018     
     76   0.0010          0.535               0.539         0.017      -0.001    0.018     
     77   0.0010          0.535               0.538         0.018      -0.001    0.018     
     78   0.0010          0.535               0.538         0.017      -0.001    0.018     
     79   0.0010          0.534               0.538         0.017      -0.001    0.018     
     80   0.0010          0.534               0.537         0.017      -0.000    0.018     
     81   0.0010          0.533               0.537         0.017      -0.001    0.018     
     82   0.0010          0.533               0.537         0.017      -0.001    0.018     
     83   0.0010          0.532               0.536         0.017      -0.000    0.018     
     84   0.0010          0.532               0.536         0.017      -0.001    0.018     
     85   0.0010          0.532             

Output()

/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/in

                                                                                           
                                                                                           
                                      Training history                                     
                                                                                           
                                                                                           
       Epoch          Training loss      Validation loss      MSE       Bias       CRPS    
 ───────────────────────────────────────────────────────────────────────────────────────── 
     #      LR            Total               Total                                        
                                                                                           
    101   0.0001          0.519               0.524         0.016      -0.001    0.018     
    102   0.0001          0.517               0.524         0.016      -0.001    0.018     
    103   0.0001          0.517               0.524         0.016      -0.001    0.018     
    104   0.0001          0.517               0.524         0.016      -0.001    0.018     
    105   0.0001          0.517               0.524         0.016      -0.001    0.018     
    106   0.0001          0.517               0.523         0.016      -0.001    0.018     
    107   0.0001          0.517               0.523         0.016      -0.001    0.018     
    108   0.0001          0.516               0.523         0.016      -0.001    0.018     
    109   0.0001          0.516               0.523         0.016      -0.001    0.018     
    110   0.0001          0.516               0.523         0.016      -0.001    0.018     
    111   0.0001          0.516               0.523         0.016      -0.001    0.018     
    112   0.0001          0.516               0.523         0.016      -0.001    0.018     
    113   0.0001          0.516               0.523         0.016      -0.001    0.018     
    114   0.0001          0.516               0.523         0.016      -0.001    0.018     
    115   0.0001          0.516               0.523         0.016      -0.001    0.018     
    116   0.0001          0.516               0.523         0.016      -0.001    0.018     
    117   0.0001          0.515               0.523         0.016      -0.001    0.018     
    118   0.0001          0.515               0.523         0.016      -0.001    0.018     
    119   0.0001          0.515               0.523         0.016      -0.001    0.018     
    120   0.0001          0.515               0.523         0.016      -0.001    0.018     
    121   0.0001          0.515               0.522         0.016      -0.001    0.018     
    122   0.0001          0.515               0.522         0.016      -0.001    0.018     
    123   0.0001          0.515               0.522         0.016      -0.001    0.018     
    124   0.0001          0.515               0.522         0.016      -0.001    0.018     
    125   0.0001          0.515               0.522         0.016      -0.001    0.018     
    126   0.0001          0.515               0.522         0.016      -0.001    0.018     
    127   0.0001          0.515               0.522         0.016      -0.001    0.018     
    128   0.0001          0.515               0.522         0.016      -0.000    0.018     
    129   0.0001          0.514               0.522         0.016      -0.000    0.018     
    130   0.0001          0.514               0.522         0.016      -0.000    0.018     
    131   0.0001          0.514               0.522         0.016      -0.000    0.018     
    132   0.0001          0.514               0.522         0.016      -0.000    0.018     
    133   0.0001          0.514               0.522         0.016      -0.000    0.018     
    134   0.0001          0.514               0.522         0.016      -0.000    0.018     
    135   0.0001          0.513             

/home/inderpreet/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FullyConnected. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/inderpreet/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/inderpreet/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FullyConnectedBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/inderpreet/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon load

In [6]:
qrnn.save(outputfile)

In [7]:
%matplotlib inline

y_pre = []
y = []
y_prior = []
y_pos_mean = []
x_in = []

nbatch = len(validation_data)
print (nbatch)
for i in range(nbatch):
    
    xx, yy = validation_data[i]
    
    x = xx.detach().numpy() 

    y_pre.append(qrnn.predict(xx)) 
    y_pos_mean.append((qrnn.posterior_mean(xx)))
       
    y.append(yy.detach().numpy())
    x_in.append(x)

x_in = np.concatenate(x_in, axis = 0)
y_pre = np.concatenate(y_pre, axis = 0)
y = np.concatenate(y, axis= 0)
y_pos_mean = np.concatenate(y_pos_mean, axis = 0)

plt.rcParams.update({'font.size': 20})
bins1 = np.arange(0, 30, 0.1)



387


TypeError: 'DataLoader' object is not subscriptable

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
bins1 = np.arange(0, 30, 0.1)

ax.hist(y_pre[:, imedian], bins1, density = True , histtype = "step", label = "predicted")


ax.hist(y, bins1, density = True, histtype = "step", label = "observed")
ax.set_yscale('log')
ax.set_xscale('log')

ax.legend()
ax.set_ylabel("PDF")
ax.set_xlabel("IWP[kg/m2]")

In [ ]:
import scipy
from matplotlib import ticker, cm
xyrange = [[0, 25], [0, 25]] # data range


bins = [50, 50] # number of bins
hh, locx, locy = np.histogram2d(y, y_pos_mean, 
                                range=xyrange, bins=bins, density = True)
posx = np.digitize(y, locx)
posy = np.digitize(y_pos_mean, locy)

fig, ax = plt.subplots(1, 1, figsize = [10, 8])
cs = ax.contourf(np.flipud(hh.T),
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
cbar = fig.colorbar(cs)
ax.set_ylim([0, 20])
ax.set_xlim([0, 20])
xy = np.arange(0, 13, 1)
yy = xy
ax.plot(xy, yy)
ax.set_ylabel("IWP Predicted [kg/m2]")
ax.set_xlabel("IWP Observed [kg/m2]")
ax.grid('on')
fig.savefig("qrnn.png", bbox_inches = "tight")
#ax.set_yscale('log')
#ax.set_xscale('log')